## Example B.3 Parent Breadth-first Search

Examples come from https://people.eecs.berkeley.edu/~aydin/GraphBLAS_API_C_v13.pdf

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import graphblas as gb
from graphblas import Matrix, Vector, Scalar
from graphblas import dtypes
from graphblas import unary, binary, monoid, semiring

In [ ]:
# Create initial data objects
edges = [
    [3, 0, 3, 5, 6, 0, 6, 1, 6, 2, 4, 1],
    [0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5, 6],
]
A = Matrix.from_coo(edges[0], edges[1], [True for _ in edges[0]])
s = 1

In [ ]:
gb.viz.draw(A)

### parent breadth-first search (BFS) in GraphBLAS
```
1 #include <stdlib.h>
2 #include <stdio.h>
3 #include <stdint.h>
4 #include <stdbool.h>
5 #include ”GraphBLAS.h”
6
7 /∗
8  ∗ Given a binary n x n adjacency matrix A and a source vertex s, performs a BFS
9  ∗ traversal of the graph and sets parents[i] to the index i's parent.
10 ∗ The parent of the root vertex, s, will be set to itself (parents[s] == s). If
11 ∗ vertex i is not reachable from s, parents[i] will not contain a stored value.
12 ∗/
13 GrB_Info BFS(GrB_Vector *v, const GrB_Matrix A, GrB_Index s)
14 {
15   GrB_Index N;
16   GrB_Matrix nrows(&N, A); // N = # of rows of A
17
18   // create index ramp for index_of() functionality
19   GrB_Index *idx = (GrB_Index *) malloc (N*sizeof(GrB_Index));
20   for (GrB_Index i = 0; i < N; ++i) idx[i] = i;
21   GrB_Vector index_ramp;
22   GrB_Vector_new(&index_ramp, GrB_UINT64, N);
23   GrB_Vector_build_UINT64(index_ramp, idx, idx, N, GrB_PLUS_INT64);
24   free(idx);
25
26   GrB_Vector_new(parents, GrB_UINT64, N);
27   GrB_Vector_setElement(*parents, s, s); // parents[s] = s
28
29   GrB_Vector wavefront;
30   GrB_Vector_new(&wavefront, GrB_UINT64, N);
31   GrB_Vector_setElement(wavefront, 1UL, s); // wavefront[s] = 1
18   GrB_Vector_new(v, GrB_INT32, N) ; // Vector<int32_t> v(n) = 0
19
33   /∗
34    ∗ BFS traversal and label the vertices.
35    ∗/
36   GrB_Index nvals;
37   GrB_Vector_nvals(&nvals, wavefront);
38   
39   while (nvals > 0) 
40   {
41     // convert all stored values in wavefront to their 0−based index
42     GrB_eWiseMult(wavefront, GrB_NULL, GrB_NULL, GrB_FIRST_UINT64,
43     index_ramp, wavefront, GrB_NULL);
44
45     // ”FIRST” because left−multiplying wavefront rows. Masking out the parent
46     // list ensures wavefront values do not overwrite parents already stored.
47     GrB_vxm(wavefront, * parents, GrB_NULL, GrB_MIN_FIRST_SEMIRING_UINT64,
48             wavefront, A, GrB_DESC_RSC);
49
50     // Don’t need to mask here since we did it in mxm. Merges new parents in
51     // current wave front with existing parents : parents += wavefront
52     GrB_apply(* parents, GrB_NULL, GrB_PLUS_UINT64,
53     GrB_IDENTITY_UINT64, wavefront, GrB_NULL);
54
55     GrB_Vector_nvals(&nvals, wavefront);
56   }
57
58   GrB_free(&wavefront);
59   GrB_free(&index_ramp);
60
61   return GrB_SUCCESS;
62 }
```

## Python implementation

In [ ]:
N = A.nrows
index_ramp = Vector(dtypes.UINT64, N)
index_ramp.build(range(N), range(N))
parents = Vector(dtypes.UINT64, N)
parents[s] << s
wavefront = Vector(dtypes.UINT64, N)
wavefront[s] << 1

In [ ]:
while wavefront.nvals > 0:
    # convert all stored values in wavefront to their 0−based index
    wavefront << index_ramp.ewise_mult(wavefront, binary.first)
    # ”FIRST” because left−multiplying wavefront rows. Masking out the parent
    # list ensures wavefront values do not overwrite parents already stored.
    wavefront(~parents.S, replace=True) << wavefront.vxm(A, semiring.min_first)
    # Don’t need to mask here since we did it in mxm. Merges new parents in
    # current wave front with existing parents : parents += wavefront
    parents(binary.plus) << wavefront
parents

Let's Step thru each loop to watch the action unfold

In [ ]:
# Only run this cell once -- it initializes things
parents.clear()
parents[s] << s
wavefront.clear()
wavefront[s] << 1

In [ ]:
print("Proceed" if wavefront.nvals > 0 else "Done")

In [ ]:
# convert all stored values in wavefront to their 0−based index
wavefront << index_ramp.ewise_mult(wavefront, binary.first)
wavefront

In [ ]:
# ”FIRST” because left−multiplying wavefront rows. Masking out the parent
# list ensures wavefront values do not overwrite parents already stored.
wavefront(~parents.S, replace=True) << wavefront.vxm(A, semiring.min_first)
wavefront

In [ ]:
# Don’t need to mask here since we did it in mxm. Merges new parents in
# current wave front with existing parents : parents += wavefront
parents(binary.plus) << wavefront
parents